In [ ]:
using Plots
using Random
using Printf
using LaTeXStrings
using Distributions
using DelimitedFiles
using LinearAlgebra

In [ ]:
# for readability, these are good settings to use
default(xtickfontsize=14,  ytickfontsize=14, ztickfontsize=14,
    guidefontsize=14, legendfontsize=12, lw=2,ms=8)

## Load Traveling Salesman Data Set
From https://github.com/pdrozdowski/TSPLib.Net/blob/master/TSPLIB95/tsp/bays29.tsp

In [ ]:
# city locations
data=DelimitedFiles.readdlm("Bays29_TSP.txt");
data = data[:,2:3];
scatter(data[:,1], data[:,2],label="Cities")
xlabel!(L"$x$")
ylabel!(L"$y$")

In [ ]:
data

## Construct Traveling Salesman Functions

In [ ]:
# partial path reversal move
function PPR(x)

    m = length(x);
    
    # find two random indices in 2,...,m and order them
    i = sample(2:m);
    j = sample(setdiff(2:m,i));
    
    k = min(i,j)
    l = max(i,j)
    
    # perform partial path reversal
    y = copy(x);
    @. y[k:l] = x[l:-1:k];
    
    return y
end

# norm between cities i and j
function dist(i,j)
   return norm(data[i,:]-data[j,:])
end

# cost function
function S(x)
    cost = 0.0;
    
    for j in 1:length(x)-1
        cost+=dist(x[j],x[j+1]);
    end
    cost+= dist(x[1],x[end]);
    return cost
end

# accept/reject function
a = (x,y,T) -> exp((S(x)-S(y))/T)

## Trial Run

In [ ]:
Random.seed!(100)
# make city 1 the initial/final city
X = 1:1:29|>collect;
n_iter = 10^2;


ΔE = 0;
for t in 1:n_iter
    Y = PPR(X);
    ζ = rand();
    ΔS += abs(S(X)-S(Y))/n_iter;
    if (ζ< a(X,Y,1))
        @. X = Y;
    end
end
println(@sprintf("Mean ΔS: %g", ΔS));

Textbook suggests using $\Delta S = 1000$, this this is quite reasonable

# SA Run

In [ ]:
Random.seed!(100)

# make city 1 the initial/final city
X = 1:1:29|>collect;
#X = [1; randperm(28).+1];
xopt = copy(X);
X_vals = [copy(X)];

n_iter = 10^5;

# these values follow those set in the text
T = t-> 1000/log(0.01 + t);

for t in 1:n_iter
    
    Y = PPR(X);
    ζ = rand();
    if (ζ< a(X,Y,T(t)))
        @. X = Y;
    end
    push!(X_vals, copy(X)); 
    if S(X)< S(xopt)
        @. xopt = X;
    end

end

println(@sprintf("Best Result = %g", S(xopt)));
println(X);

In [ ]:
plot(1:n_iter+1,S.(X_vals), label=L"$S_n$",lw=2, xscale=:log10)
plot!(1:n_iter+1,1 ./(1:n_iter+1) .* cumsum(S.(X_vals)), label="Running Avg.", lw=2)
xlabel!(L"$n$")

In [ ]:
scatter(data[:,1], data[:,2],label="Cities")
scatter!([data[1,1]], [data[1,2]],label="Starting City", marker=:x)
# plot the path
for j in 1:length(xopt)-1
    plot!([data[xopt[j],1], data[xopt[j+1],1]],[data[xopt[j],2], data[xopt[j+1],2]],lw=2,label="", color=:red)
end
plot!([data[xopt[end],1], data[xopt[1],1]],[data[xopt[end],2], data[xopt[1],2]],lw=2,label="", color=:red)
xlabel!(L"$x$")
ylabel!(L"$y$")
title!("Optimal Run")

In [ ]:
anim = @animate for i = 1:10^3+1
    scatter(data[:,1], data[:,2],label="Cities")
    scatter!([data[1,1]], [data[1,2]],label="Starting City", marker=:x)
    for j in 1:length(xopt)-1
        plot!([data[X_vals[i][j],1], data[X_vals[i][j+1],1]],
            [data[X_vals[i][j],2], data[X_vals[i][j+1],2]],lw=2,label="", color=:red)
    end
    plot!([data[X_vals[i][end],1], data[X_vals[i][1],1]],
        [data[X_vals[i][end],2], data[X_vals[i][1],2]],lw=2,label="", color=:red)
    title!(@sprintf("Iteration = %d", i-1));
end

In [ ]:
gif(anim,fps=60)